In [1]:
from transformers import Wav2Vec2Tokenizer, Wav2Vec2Model, Trainer, TrainingArguments
import transformers
import soundfile as sf
import torch
from torch import nn
import pandas as pd
import copy
import math
import librosa
from pathlib import Path
import torch.utils.data as data
import torch.optim as optim
from torch.optim import lr_scheduler
import os
import random
import glob
from tqdm import tqdm
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Running on device: {}'.format(device))
print(torch.cuda.get_device_name(0))

Running on device: cuda
NVIDIA GeForce RTX 2080 Ti


In [2]:
from transformers import AutoProcessor, AutoModelForCTC
# processor = AutoProcessor.from_pretrained("ydshieh/wav2vec2-large-xlsr-53-chinese-zh-cn-gpt")
# model = AutoModelForCTC.from_pretrained("ydshieh/wav2vec2-large-xlsr-53-chinese-zh-cn-gpt")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")
print(model)
model.to(device)


C:\Users\LaylarZhang\anaconda3\envs\emotion\lib\site-packages\transformers\configuration_utils.py:349: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['quantizer.weight_proj.weight', 'project_hid.weight', 'project_q.weight', 'project_hid.bias', 'quantizer.weight_proj.bias', 'project_q.bias', 'quantizer.codevectors']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model

Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureExtractor(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
      )
      (2): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
      )
      (3): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
      )
      (4): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
      )
      (5): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
      )
      (6): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), 

Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureExtractor(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
      )
      (2): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
      )
      (3): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
      )
      (4): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
      )
      (5): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
      )
      (6): Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), 

In [3]:
DATASET_PATH = '../dataset/chinese_dataset/'
emotions = ['Angry', 'Happy', 'Neutral', 'Sad']
OUTPUT = '../models/eng_chinese/'

BATCHSIZE = 20
EPOCHS = 30
optimizer = optim.Adam(model.parameters())
scheduler = lr_scheduler.StepLR(optimizer, 5)

## Load dataset


In [4]:
all_dataset = []
for i, emotion in enumerate(emotions):
    filepath = Path(DATASET_PATH + emotion + '/')
    for file in filepath.glob("*.wav"): 
        wav, rate = librosa.load(file, sr=16000, mono=True, res_type="kaiser_fast")
        tensor = torch.tensor(wav, device="cuda")
        all_dataset.append((tensor, i))
print(len(all_dataset[10][0]))
print(len(all_dataset))

75520
160


In [5]:
root = glob.glob(DATASET_PATH + "*")
for i, filepath in enumerate(root):
    filepath = filepath.replace("\\", "/")
    all_wav = os.listdir(filepath)
    wav_sample = random.sample(all_wav, 10)

    for sample in tqdm(wav_sample):
        file = f"{filepath}/{sample}"
        # add noise
        wav, rate = librosa.load(file, sr=16000, mono=True, res_type="kaiser_fast")
        wav_n = wav + 0.009*np.random.normal(0,1,len(wav))
        tensor = torch.tensor(wav, device="cuda")
        all_dataset.append((tensor, i))

print(len(all_dataset[10][0]))
print(len(all_dataset))

100%|██████████| 10/10 [00:00<00:00, 23.76it/s]

75520
200


In [6]:
def sort_data(data_set):
    indices = sorted(range(len(data_set)),
                     key=lambda k: len(data_set[k][0]),
                     reverse=True)
    data_set = [data_set[i] for i in indices]
    return data_set, indices

In [7]:
def split_data(all_dataset):
    # Split dataset
    data_size = len(all_dataset)
    dataset_sizes = {
    "train": int(data_size*0.8),
    "test": int(data_size*0.2),
    }

    train_set, test_set = data.random_split(all_dataset, [dataset_sizes["train"], dataset_sizes["test"]])
    # sort
    sorted_train, train_indices = sort_data(train_set)
    sorted_test, test_indices = sort_data(test_set)

    def pad_tensor(vec, pad, dim):
        pad_size = list(vec.shape)
        pad_size[dim] = pad - vec.size(dim)
        return torch.cat([vec, torch.zeros(*pad_size).cuda()], dim=dim)

    def collate_fn(instances):
        
        max_len = max(map(lambda x: x[0].shape[0], instances))
        batch = []
        for (x,y) in instances:
            batch.append((pad_tensor(x, pad=max_len, dim=-1), y))
        
        f = list(map(lambda x: x[0], batch))
        l = list(map(lambda x: x[1], batch))
        features = torch.stack(f, dim=0)
        labels = torch.Tensor(l)
        return (features, labels)

    dataloaders = {
        "train": torch.utils.data.DataLoader(sorted_train, batch_size=BATCHSIZE,  num_workers=0, drop_last=True, collate_fn=collate_fn),
        "test": torch.utils.data.DataLoader(sorted_test, batch_size=BATCHSIZE,  num_workers=0, drop_last=True, collate_fn=collate_fn),
    }

    return dataloaders, dataset_sizes

In [8]:
dataloaders, dataset_sizes = split_data(all_dataset)
print(dataset_sizes["train"], dataset_sizes["test"])

160 40


In [9]:
class TranferModel(nn.Module):

  def __init__(self, wav2vec, n_classes):
    super().__init__()
    self.wav2vec = wav2vec
    self.dropout = nn.Dropout(0.2)
    self.linear = nn.Linear(768, n_classes)
    self.device = device
    self.to(self.device)

  def forward(self, input_values, attention_mask=None):
    x = self.wav2vec(input_values=input_values, attention_mask=attention_mask)
    # print(x.last_hidden_state.shape)
    x = x.last_hidden_state.mean(1)
    # print(x.shape)
    x = self.dropout(x)
    x = self.linear(x)
    return x

In [10]:
def train (model, dataloaders, dataset_sizes, num_epochs, optimizer, scheduler, checkpoint=None):
    criterion = nn.CrossEntropyLoss()

    outputlist = {'train': {'loss':[], 'acc': []}, 'test': {'loss':[], 'acc': []}}

    if checkpoint is None:
        best_model_wts = copy.deepcopy(model.state_dict())
        best_loss = math.inf
        best_acc = 0.0
    else:
        print(
            f'Test loss: {checkpoint["best_test_loss"]}, Test accuracy: {checkpoint["best_test_accuracy"]}')
        model.load_state_dict(checkpoint['model_state_dict'])
        best_model_wts = copy.deepcopy(model.state_dict())
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        best_loss = checkpoint['best_test_loss']
        best_acc = checkpoint['best_test_accuracy']

    #start training
    for epoch in range(1, num_epochs+1):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('--' * 10)

        for phase in ["train", "test"]:
            running_loss = 0.0
            running_corrects = 0

            if phase == "train":
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            
            for i, (inputs, labels) in enumerate(dataloaders[phase]):
                
                inputs = inputs.to(device)
                labels = torch.tensor([i.long() for i in labels])
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    # print("outputs: ", outputs)
                    _, preds = torch.max(outputs, 1)
                    # print("preds: ", preds)
                    loss = criterion(outputs, labels)

                    # backward propogation and optimize in 'train' mode
                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == "train":
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            #record the loss and accuracy for visualization
            outputlist[phase]['loss'].append(epoch_loss)
            outputlist[phase]['acc'].append(epoch_acc.cpu())

            print('{} Loss: {:.4f} Accuracy: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # output test result
            if phase == "test" and epoch_acc > best_acc:
                print(f'New best model found!')
                print(f'New record accuracy: {epoch_acc}, Previous record accuracy: {best_acc}')
                best_loss = epoch_loss
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                if epoch_acc > 0.40: 
                    check_point_path = OUTPUT + "epoch" + str(epoch) + ".pth"

                    torch.save({'model_state_dict': model.state_dict(),
                                'optimizer_state_dict': optimizer.state_dict(),
                                'best_test_loss': best_loss,
                                'best_test_accuracy': best_acc,
                                'scheduler_state_dict': scheduler.state_dict(),
                                }, check_point_path)

    print('Best test Accuracy: {:.4f} Best test loss: {:.4f}'.format(best_acc, best_loss))

    # load and return the best model weights
    model.load_state_dict(best_model_wts)
    return model, best_loss, best_acc, outputlist
            

In [11]:
new_model = TranferModel(model, len(emotions))

best_model, best_test_loss, best_test_acc, outputlist = train(
    new_model, dataloaders, dataset_sizes, EPOCHS, optimizer, scheduler
)

Epoch 1/30
--------------------
train Loss: 1.6533 Accuracy: 0.2563
test Loss: 1.3851 Accuracy: 0.3250
New best model found!
New record accuracy: 0.325, Previous record accuracy: 0.0
Epoch 2/30
--------------------
train Loss: 1.4892 Accuracy: 0.1938
test Loss: 1.4611 Accuracy: 0.2000
Epoch 3/30
--------------------
train Loss: 1.4265 Accuracy: 0.2375
test Loss: 1.3931 Accuracy: 0.2250
Epoch 4/30
--------------------
train Loss: 1.4074 Accuracy: 0.2000
test Loss: 1.4039 Accuracy: 0.2000
Epoch 5/30
--------------------
train Loss: 1.3996 Accuracy: 0.2687
test Loss: 1.4062 Accuracy: 0.2000
Epoch 6/30
--------------------
train Loss: 1.3851 Accuracy: 0.2750
test Loss: 1.4053 Accuracy: 0.2000
Epoch 7/30
--------------------
train Loss: 1.3840 Accuracy: 0.2188
test Loss: 1.4037 Accuracy: 0.2000
Epoch 8/30
--------------------
train Loss: 1.3844 Accuracy: 0.2750
test Loss: 1.4019 Accuracy: 0.2000
Epoch 9/30
--------------------
train Loss: 1.3879 Accuracy: 0.2625
test Loss: 1.4003 Accuracy: 